# Do Blue Zone diets work in Berkeley?

##### Blue Zones are areas of the world where people live much longer than the rest of the population and there are only 5 Blue Zones in the world in countries such as Greece, Italy, Japan, USA, and Costa Rica. The longevity of the population is a reflection of the their culture, community, and most importantly diets since they all tend to lead healthy lifestyles. 
##### Our group wants to see how feasible it would be to eat a Blue Zone diet as Berkeley students as compared to our typical diets. To do this, we are testing a typical Mediterranean diet, eaten in Ikaria, Greece, and an Okinawa, Japan diet against a typical Berkeley student's diet to see which has the lowest price (with Berkeley Safeway prices) and most nutritional value.

### Dietary Reference Intakes

In [1]:
#checking that we are in the correct working directory
!pwd

#installing neccesary packages and access to fdc data 

!pip install -r requirements.txt #--upgrade

from  scipy.optimize import linprog as lp
import numpy as np
import pandas as pd

import fooddatacentral as fdc
import warnings

/home/jovyan/EEP153_Materials/Project2


In [2]:
#Read diet minimum data
diet_min = pd.read_csv("diet_minimums.csv")
#drop unneeded columns 
diet_min = diet_min.drop(columns=["Unnamed: 0"])
diet_min = diet_min.set_index('Nutrition')

diet_min
#set "Nutrition" as the index of the Dataframe 
#diet_min = diet_min.set_index('Nutrition')  

,Source,C 1-3,F 4-8,M 4-8,F 9-13,M 9-13,F 14-18,M 14-18,F 19-30,M 19-30,F 31-50,M 31-50,F 51+,M 51+
Nutrition,,,,,,,,,,,,,,
Energy,---,1000.0,1200.0,1400.0,1600.0,1800.0,1800.0,2200.0,2000.0,2400.0,1800.0,2200.0,1600.0,2000.0
Protein,RDA,13.0,19.0,19.0,34.0,34.0,46.0,52.0,46.0,56.0,46.0,56.0,46.0,56.0
"Fiber, total dietary",---,14.0,16.8,19.6,22.4,25.2,25.2,30.8,28.0,33.6,25.2,30.8,22.4,28.0
"Folate, DFE",RDA,150.0,200.0,200.0,300.0,300.0,400.0,400.0,400.0,400.0,400.0,400.0,400.0,400.0
"Calcium, Ca",RDA,700.0,1000.0,1000.0,1300.0,1300.0,1300.0,1300.0,1000.0,1000.0,1000.0,1000.0,1200.0,1000.0
"Carbohydrate, by difference",RDA,130.0,130.0,130.0,130.0,130.0,130.0,130.0,130.0,130.0,130.0,130.0,130.0,130.0
"Iron, Fe",RDA,7.0,10.0,10.0,8.0,8.0,15.0,11.0,18.0,8.0,18.0,8.0,8.0,8.0
"Magnesium, Mg",RDA,80.0,130.0,130.0,240.0,240.0,360.0,410.0,310.0,400.0,320.0,420.0,320.0,420.0
Niacin,RDA,6.0,8.0,8.0,12.0,12.0,14.0,16.0,14.0,16.0,14.0,16.0,14.0,16.0


In [3]:
def dietary_ref_intake(age,sex,df):
    """Takes in age and sex, and returns the dietary reference intake for the chosen population"""

    if age <= 3:
        col = 'C 1-3'
    age_ranges = [(4,8),(9,13),(14,18),(19,30),(31,50),(50,100)]
    for age_range in age_ranges:
        if age >= age_range[0] and age <= age_range[1]:
            col = sex + ' ' + str(age_range[0]) + '-' + str(age_range[1])
    return pd.Series(df[col]) 

In [4]:
# Example of minimum dietary requirements for a male aged 19
dietary_ref_intake(age=19,sex='F',df=diet_min)

Nutrition
Energy                            2000.0
Protein                             46.0
Fiber, total dietary                28.0
Folate, DFE                        400.0
Calcium, Ca                       1000.0
Carbohydrate, by difference        130.0
Iron, Fe                            18.0
Magnesium, Mg                      310.0
Niacin                              14.0
Phosphorus, P                      700.0
Potassium, K                      4700.0
Riboflavin                           1.1
Thiamin                              1.1
Vitamin A, RAE                     700.0
Vitamin B-12                         2.4
Vitamin B-6                          1.3
Vitamin C, total ascorbic acid      75.0
Vitamin E (alpha-tocopherol)        15.0
Vitamin K (phylloquinone)           90.0
Zinc, Zn                             8.0
Name: F 19-30, dtype: float64

## Function to Solve Lowest Cost

In [5]:
def solve_subsistence_problem(FoodNutrients,Prices,dietmin,dietmax,max_weight=None,tol=1e-6):
    """Solve Stigler's Subsistence Cost Problem.

    Inputs:
       - FoodNutrients : A pd.DataFrame with rows corresponding to foods, columns to nutrients.
       - Prices : A pd.Series of prices for different foods
       - diet_min : A pd.Series of DRIs, with index corresponding to columns of FoodNutrients,
                    describing minimum intakes.
       - diet_max : A pd.Series of DRIs, with index corresponding to columns of FoodNutrients,
                    describing maximum intakes.
       - max_weight : Maximum weight (in hectograms) allowed for diet.
       - tol : Solution values smaller than this in absolute value treated as zeros.
       
    """
    try: 
        p = Prices.apply(lambda x:x.magnitude)
    except AttributeError:  # Maybe not passing in prices with units?
        warnings.warn("Prices have no units.  BE CAREFUL!  We're assuming prices are per hectogram or deciliter!")
        p = Prices

    p = p.dropna()

    # Compile list that we have both prices and nutritional info for; drop if either missing
    use = p.index.intersection(FoodNutrients.columns)
    p = p[use]

    # Drop nutritional information for foods we don't know the price of,
    # and replace missing nutrients with zeros.
    Aall = FoodNutrients[p.index].fillna(0)

    # Drop rows of A that we don't have constraints for.
    Amin = Aall.loc[Aall.index.intersection(dietmin.index)]
    Amin = Amin.reindex(dietmin.index,axis=0)
    idx = Amin.index.to_frame()
    idx['type'] = 'min'
    #Amin.index = pd.MultiIndex.from_frame(idx)
    #dietmin.index = Amin.index
    
    Amax = Aall.loc[Aall.index.intersection(dietmax.index)]
    Amax = Amax.reindex(dietmax.index,axis=0)
    idx = Amax.index.to_frame()
    idx['type'] = 'max'
    #Amax.index = pd.MultiIndex.from_frame(idx)
    #dietmax.index = Amax.index

    # Minimum requirements involve multiplying constraint by -1 to make <=.
    A = pd.concat([Amin,
                   -Amax])

    b = pd.concat([dietmin,
                   -dietmax]) # Note sign change for max constraints

    # Make sure order of p, A, b are consistent
    A = A.reindex(p.index,axis=1)
    A = A.reindex(b.index,axis=0)

    if max_weight is not None:
        # Add up weights of foods consumed
        A.loc['Hectograms'] = -1
        b.loc['Hectograms'] = -max_weight
        
    # Now solve problem!  (Note that the linear program solver we'll use assumes
    # "less-than-or-equal" constraints.  We can switch back and forth by
    # multiplying $A$ and $b$ by $-1$.)

    result = lp(p, -A, -b, method='highs')

    result.A = A
    result.b = b
    
    if result.success:
        result.diet = pd.Series(result.x,index=p.index)
    else: # No feasible solution?
        warnings.warn(result.message)
        result.diet = pd.Series(result.x,index=p.index)*np.nan  

    return result

## Generic Berkeley Student Diet

In [6]:
from  scipy.optimize import linprog as lp
import numpy as np
import warnings

In [7]:
apikey = "sCD07VKZEF2pe7ewJNYSSWlOHY0nRMda34HLcp80"

In [8]:
%pip install pandas
%pip install gnupg

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


#### Prices for Generic Berkeley Diet

In [9]:
SHEETs = [# BERKELEY DIET foods, Berkeley prices
          ("https://docs.google.com/spreadsheets/d/11Ou4aZ8bE12J6dY9hmyUeCFFCNpplexnOGtfJVKdgbY/edit#gid=628663795","GENERIC"),
         ]

In [10]:
import pandas as pd
from eep153_tools.sheets import read_sheets

df = read_sheets(SHEETs[0][0])[SHEETs[0][1]]
df
df['FDC'] = pd.to_numeric(df['FDC'], errors='coerce').fillna(0).astype(int)

print(df)

Key available for students@eep153.iam.gserviceaccount.com.


/srv/conda/lib/python3.9/site-packages/eep153_tools/sheets.py:150: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  D[w.title]=df.apply(lambda x: pd.to_numeric(x,errors='ignore'))


        FDC                     Food  Quantity   Units  Price                  \
0   2646170           Chicken Breast       1.0     lbs   4.99 NaN NaN NaN NaN   
1   2100593           Chicken Thighs       1.0     lbs   2.99 NaN NaN NaN NaN   
2   1990910     Ground Chicken (96%)       1.0     lbs   8.69 NaN NaN NaN NaN   
3   2033779      Ground Turkey (93%)       1.0     lbs   3.99 NaN NaN NaN NaN   
4   2546569     Smoked Turkey Breast       1.0     lbs  10.99 NaN NaN NaN NaN   
5   2641764    Roasted Turkey Breast       1.0     lbs   5.00 NaN NaN NaN NaN   
6   2646168  Boneless Pork Loin chop       1.0     lbs   8.99 NaN NaN NaN NaN   
7   1942314        Ground Beef (80%)       1.0     lbs   4.99 NaN NaN NaN NaN   
8   2157166                    Steak       1.0     lbs   6.99 NaN NaN NaN NaN   
9   2341312               Lamb Chops       1.0     lbs  12.99 NaN NaN NaN NaN   
10  2064258              Pink Salmon       1.0     lbs   5.99 NaN NaN NaN NaN   
11  2486028          Atlanti

#### Nutritional Information for Berkeley Diet Foods

In [ ]:
import fooddatacentral as fdc
import warnings

D = {}
count = 0
for food in df.Food.tolist():
    try:
        FDC = df.loc[df.Food==food,:].FDC[count]
        count+=1
        D[food] = fdc.nutrients(apikey,FDC).Quantity
    except AttributeError: 
        warnings.warn("Couldn't find FDC Code %s for food %s." % (food, FDC))        

FoodNutrients = pd.DataFrame(D,dtype=float)
FoodNutrients

In [14]:
# Unit Conversion
# Convert food quantities to FDC units
df['FDC Quantity'] = df[['Quantity','Units']].T.apply(lambda x : fdc.units(x['Quantity'],x['Units']))

# Now may want to filter df by time or place--need to get a unique set of food names.
df['FDC Price'] = df['Price']/df['FDC Quantity']

df.dropna(how='any') # Drop food with any missing data

# To use minimum price observed
Prices = df.groupby('Food',sort=False)['FDC Price'].min()

/srv/conda/lib/python3.9/site-packages/pandas/core/dtypes/cast.py:1598: UnitStrippedWarning: The unit of the quantity is stripped when downcasting to ndarray.
  result[:] = values


#### Berkeley Result

In [15]:
group = 'M 19-30'
tol = 1e-6

result = solve_subsistence_problem(FoodNutrients,Prices,diet_min[group],diet_max[group],tol=tol)

print("Cost of diet for %s is $%4.2f per day.\n" % (group,result.fun))

# Put back into nice series
diet = result.diet

print("\nDiet (in 100s of grams or milliliters):")
print(diet[diet >= tol])  # Drop items with quantities less than precision of calculation.
print()

tab = pd.DataFrame({"Outcome":np.abs(result.A).dot(diet),"Recommendation":np.abs(result.b)})
print("\nWith the following nutritional outcomes of interest:")
print(tab)
print()

print("\nConstraining nutrients are:")
excess = tab.diff(axis=1).iloc[:,1]
print(excess.loc[np.abs(excess) < tol*100].index.tolist())

NameError: name 'diet_max' is not defined

## Mediterranean Diet

#### Prices for Mediterranean Diet

In [16]:
mSHEETs = [# MEDITERRANEAN foods, Berkeley prices
          ("https://docs.google.com/spreadsheets/d/11Ou4aZ8bE12J6dY9hmyUeCFFCNpplexnOGtfJVKdgbY/edit#gid=628663795","MED"),
         ]

In [17]:
import pandas as pd
from eep153_tools.sheets import read_sheets

mdf = read_sheets(mSHEETs[0][0])[mSHEETs[0][1]]
mdf
mdf['FDC'] = pd.to_numeric(mdf['FDC'], errors='coerce').fillna(0).astype(int)

print(mdf)

Key available for students@eep153.iam.gserviceaccount.com.


/srv/conda/lib/python3.9/site-packages/eep153_tools/sheets.py:150: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  D[w.title]=df.apply(lambda x: pd.to_numeric(x,errors='ignore'))


        FDC                              Food  Quantity Units  Price
0   2617829                           Spinach       1.0   lbs   3.99
1   2345103                              Kale       1.0   lbs   3.99
2   2345103                           Arugula       1.0   lbs   1.99
3   2345315                         Red onion       1.0   lbs   2.49
4   2345173                           Carrots       1.0   lbs   1.49
5   2344876                            Potato       1.0   lbs   0.80
6   2345212              Sweet potato (baked)       1.0   lbs   1.99
7   2345298                            Celery       1.0   lbs   2.99
8   2415908                              Peas       1.0   lbs   2.72
9   2345357  Asparagus (cooked, no added fat)       1.0   lbs   2.99
10  2155640                              Corn       1.0   lbs   2.08
11  2345304                          Cucumber       1.0   lbs   2.99
12   169291                          Zucchini       1.0   lbs   2.99
13  2058464                       

In [18]:
from eep153_tools.sheets import read_sheets

DRI_url = "https://docs.google.com/spreadsheets/d/1y95IsQ4HKspPW3HHDtH7QMtlDA66IUsCHJLutVL-MMc/"

DRIs = read_sheets(DRI_url)

# Define *minimums*
diet_min = DRIs['diet_minimums'].set_index('Nutrition')

# Define *maximums*
diet_max = DRIs['diet_maximums'].set_index('Nutrition')

Key available for students@eep153.iam.gserviceaccount.com.


/srv/conda/lib/python3.9/site-packages/eep153_tools/sheets.py:150: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  D[w.title]=df.apply(lambda x: pd.to_numeric(x,errors='ignore'))


#### Nutritional Information for Med Diet

In [19]:
import fooddatacentral as fdc
import warnings

D = {}
count = 0
for food in mdf.Food.tolist():
    try:
        FDC = mdf.loc[mdf.Food==food,:].FDC[count]
        count+=1
        D[food] = fdc.nutrients(apikey,FDC).Quantity
    except AttributeError: 
        warnings.warn("Couldn't find FDC Code %s for food %s." % (food, FDC))        

FoodNutrients = pd.DataFrame(D,dtype=float)
FoodNutrients

,Spinach,Kale,Arugula,Red onion,Carrots,Potato,Sweet potato (baked),Celery,Peas,"Asparagus (cooked, no added fat)",...,Sign. Select Sunflower Seeds,"Almonds, raw unsalted",Diamond Walnuts,EGGS,Extra Virgin Olive Oil,kidney beans,lowfat milk,o organics spring mix,avocados,nonfat plain greek yogurt
Alanine,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.350
"Alcohol, ethyl",NaN,0.00,0.00,0.00,0.00,0.00,0.00,0.00,NaN,0.00,...,NaN,0.00,NaN,NaN,NaN,0.00,NaN,NaN,0.00,NaN
Amino acids,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000
Arginine,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.363
Ash,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.720
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Vitamin K (Menaquinone-4),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000
Vitamin K (phylloquinone),482.4,390.00,390.00,0.40,13.20,3.90,2.30,29.30,NaN,44.60,...,NaN,3.30,NaN,NaN,NaN,6.80,NaN,NaN,21.00,0.000
Vitamins and Other Components,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000
Water,NaN,89.60,89.60,90.30,88.30,72.10,75.50,95.40,NaN,92.40,...,NaN,2.34,NaN,NaN,NaN,63.30,NaN,NaN,73.20,85.000


In [20]:
# Unit Conversion
# Convert food quantities to FDC units
mdf['FDC Quantity'] = mdf[['Quantity','Units']].T.apply(lambda x : fdc.units(x['Quantity'],x['Units']))

# Now may want to filter df by time or place--need to get a unique set of food names.
mdf['FDC Price'] = mdf['Price']/mdf['FDC Quantity']

mdf.dropna(how='any') # Drop food with any missing data

# To use minimum price observed
Prices = mdf.groupby('Food',sort=False)['FDC Price'].min()

/srv/conda/lib/python3.9/site-packages/pandas/core/dtypes/cast.py:1598: UnitStrippedWarning: The unit of the quantity is stripped when downcasting to ndarray.
  result[:] = values


In [21]:
from eep153_tools.sheets import read_sheets

DRI_url = "https://docs.google.com/spreadsheets/d/1y95IsQ4HKspPW3HHDtH7QMtlDA66IUsCHJLutVL-MMc/"

DRIs = read_sheets(DRI_url)

# Define *minimums*
diet_min = DRIs['diet_minimums'].set_index('Nutrition')

# Define *maximums*
diet_max = DRIs['diet_maximums'].set_index('Nutrition')

Key available for students@eep153.iam.gserviceaccount.com.


/srv/conda/lib/python3.9/site-packages/eep153_tools/sheets.py:150: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  D[w.title]=df.apply(lambda x: pd.to_numeric(x,errors='ignore'))


#### Mediterranean Result

In [22]:
group = 'M 19-30'
tol = 1e-6

result = solve_subsistence_problem(FoodNutrients,Prices,diet_min[group],diet_max[group],tol=tol)

print("Cost of diet for %s is $%4.2f per day.\n" % (group,result.fun))

# Put back into nice series
diet = result.diet

print("\nDiet (in 100s of grams or milliliters):")
print(diet[diet >= tol])  # Drop items with quantities less than precision of calculation.
print()

tab = pd.DataFrame({"Outcome":np.abs(result.A).dot(diet),"Recommendation":np.abs(result.b)})
print("\nWith the following nutritional outcomes of interest:")
print(tab)
print()

print("\nConstraining nutrients are:")
excess = tab.diff(axis=1).iloc[:,1]
print(excess.loc[np.abs(excess) < tol*100].index.tolist())

Cost of diet for M 19-30 is $6.81 per day.


Diet (in 100s of grams or milliliters):
Arugula                      2.419039
Carrots                      0.272117
Potato                       4.545550
Almonds, raw unsalted        0.837051
kidney beans                 4.091684
nonfat plain greek yogurt    3.428571
dtype: float64


With the following nutritional outcomes of interest:
                                    Outcome  Recommendation
Nutrition                                                  
Energy                          2626.096640          2400.0
Protein                          103.720924            56.0
Fiber, total dietary              58.649977            33.6
Folate, DFE                      400.000000           400.0
Calcium, Ca                     1440.749138          1000.0
Carbohydrate, by difference      236.648705           130.0
Iron, Fe                          17.278414             8.0
Magnesium, Mg                    660.297749           400.0
Niacin           

## Okinawa Diet

#### Prices for Okinawa Diet

In [23]:
oSHEETs = [# OKINAWA foods, Berkeley prices
          ("https://docs.google.com/spreadsheets/d/11Ou4aZ8bE12J6dY9hmyUeCFFCNpplexnOGtfJVKdgbY/edit#gid=628663795","OKINAWA"),
         ]

In [24]:
import pandas as pd
from eep153_tools.sheets import read_sheets

odf = read_sheets(oSHEETs[0][0])[oSHEETs[0][1]]
odf
odf['FDC'] = pd.to_numeric(odf['FDC'], errors='coerce').fillna(0).astype(int)

print(odf)

Key available for students@eep153.iam.gserviceaccount.com.


/srv/conda/lib/python3.9/site-packages/eep153_tools/sheets.py:150: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  D[w.title]=df.apply(lambda x: pd.to_numeric(x,errors='ignore'))


        FDC               Food  Quantity Units   Price
0    451884     Sweet potatoes         1   lbs    1.99
1   2345512            Seaweed         1   lbs   79.84
2   2029705               Kelp         1   lbs  184.00
3   2029502      Bamboo shoots         1   lbs   19.36
4   2345503     Daikon raddish         1   lbs    2.49
5   1548192       Bitter melon         1   lbs    5.99
6    169975            Cabbage         1   lbs    1.49
7   2079038            Carrots         1   lbs    1.49
8    169260       Chinese okra         1   lbs    6.72
9   2653425            Pumpkin         1   lbs    4.64
10   169926       Green papaya         1   lbs    3.99
11  2343861             Millet         1   lbs   17.45
12  2343200              Wheat         1   lbs    4.00
13   356554               Rice         1   lbs    0.80
14  2008214  Buckwheat noodles         1   lbs    7.52
15  2294522               Tofu         1   lbs    3.36
16  2342914               Miso         1   lbs    6.72
17  234291

#### Nutritional Information for Okinawa Diet

In [ ]:
import fooddatacentral as fdc
import warnings

D = {}
count = 0
for food in odf.Food.tolist():
    try:
        FDC = odf.loc[odf.Food==food,:].FDC[count]
        count+=1
        D[food] = fdc.nutrients(apikey,FDC).Quantity
    except AttributeError: 
        warnings.warn("Couldn't find FDC Code %s for food %s." % (food, FDC))        

FoodNutrients = pd.DataFrame(D,dtype=float)
FoodNutrients

In [ ]:
# Unit Conversion
# Convert food quantities to FDC units
odf['FDC Quantity'] = odf[['Quantity','Units']].T.apply(lambda x : fdc.units(x['Quantity'],x['Units']))

# Now may want to filter df by time or place--need to get a unique set of food names.
odf['FDC Price'] = odf['Price']/odf['FDC Quantity']

odf.dropna(how='any') # Drop food with any missing data

# To use minimum price observed
Prices = odf.groupby('Food',sort=False)['FDC Price'].min()

In [ ]:
from eep153_tools.sheets import read_sheets

DRI_url = "https://docs.google.com/spreadsheets/d/1y95IsQ4HKspPW3HHDtH7QMtlDA66IUsCHJLutVL-MMc/"

DRIs = read_sheets(DRI_url)

# Define *minimums*
diet_min = DRIs['diet_minimums'].set_index('Nutrition')

# Define *maximums*
diet_max = DRIs['diet_maximums'].set_index('Nutrition')

#### Okinawa Result

In [32]:
group = 'M 19-30'
tol = 1e-6

result = solve_subsistence_problem(FoodNutrients,Prices,diet_min[group],diet_max[group],tol=tol)

print("Cost of diet for %s is $%4.2f per day.\n" % (group,result.fun))

# Put back into nice series
diet = result.diet

print("\nDiet (in 100s of grams or milliliters):")
print(diet[diet >= tol])  # Drop items with quantities less than precision of calculation.
print()

tab = pd.DataFrame({"Outcome":np.abs(result.A).dot(diet),"Recommendation":np.abs(result.b)})
print("\nWith the following nutritional outcomes of interest:")
print(tab)
print()

print("\nConstraining nutrients are:")
excess = tab.diff(axis=1).iloc[:,1]
print(excess.loc[np.abs(excess) < tol*100].index.tolist())

/tmp/ipykernel_283/2287643672.py:74: UserWarning: The problem is infeasible. (HiGHS Status 8: model_status is Infeasible; primal_status is At lower/fixed bound)
  warnings.warn(result.message)


TypeError: must be real number, not NoneType